# Hands-on - Procesamiento de Streaming para ML con Apache Kafka y Python

En este hands-on, exploraremos cómo trabajar con flujos de datos en tiempo real utilizando Apache Kafka como broker de mensajes y Python para el procesamiento. Simularemos un escenario donde se generan eventos de datos, se ingieren a través de Kafka, y luego se procesan en tiempo real para realizar inferencias con un modelo de Machine Learning pre-entrenado.

## 1. Requisitos Previos

Asegúrate de tener instalados los siguientes componentes en tu sistema:

* **Docker y Docker Compose:** Para levantar el servicio de Kafka de manera sencilla.
    * [Instalar Docker](https://docs.docker.com/get-docker/)
    * [Instalar Docker Compose](https://docs.docker.com/compose/install/)
* **Python 3.8+:** Para ejecutar los scripts de productor y consumidor.
    * [Descargar Python](https://www.python.org/downloads/)
* **Poetry:** Para gestionar las dependencias del proyecto Python.
    * [Instalar Poetry](https://python-poetry.org/docs/#installation)

## 2. Estructura del Proyecto

Vamos a organizar los archivos de la siguiente manera. Crea una carpeta principal para el hands-on, por ejemplo, `hands_on_streaming`. Dentro de ella, crearemos los archivos.

```
streaming1/
├── docker-compose.yml
├── producer.py
├── consumer.py
├── ml_model/
│   └── simple_model.pkl
├── pyproject.toml
└── poetry.lock
```

## 3. Configuración del Entorno (Kafka con Docker Compose)

Primero, crearemos el archivo `docker-compose.yml` para levantar un servidor Kafka y Zookeeper.

**Paso 3.1: Crear el archivo `docker-compose.yml`**

Crea un archivo llamado `docker-compose.yml` en la raíz de tu carpeta `streaming1/` y pega el siguiente contenido:

In [ ]:
version: '3.8'

services:
  zookeeper:
    image: confluentinc/cp-zookeeper:7.0.1
    hostname: zookeeper
    container_name: zookeeper
    ports:
      - "2181:2181"
    environment:
      ZOOKEEPER_CLIENT_PORT: 2181
      ZOOKEEPER_TICK_TIME: 2000

  kafka:
    image: confluentinc/cp-kafka:7.0.1
    hostname: kafka
    container_name: kafka
    ports:
      - "9092:9092"
      - "9094:9094" # Puerto para conexión externa
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: 'zookeeper:2181'
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://kafka:9092,PLAINTEXT_HOST://localhost:9094
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_INTER_BROKER_LISTENER_NAME: PLAINTEXT
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
    depends_on:
      - zookeeper

**Paso 3.2: Levantar Kafka**

Abre una terminal (no desde Jupyter, ya que necesitamos un proceso en segundo plano), navega hasta la carpeta `streaming1/` y ejecuta el siguiente comando para levantar los servicios de Kafka y Zookeeper en segundo plano:

```bash
docker compose up -d
```

Puedes verificar que los contenedores estén corriendo con `docker compose ps`.

## 4. Preparación del Modelo de Machine Learning

Para este hands-on, usaremos un modelo muy simple pre-entrenado.

**Paso 4.1: Crear la carpeta `ml_model`**

Puedes crearla directamente con el siguiente comando en una celda de código (asegúrate de que el directorio `streaming1/ml_model` exista):


In [ ]:
import os

os.makedirs('streaming1/ml_model', exist_ok=True)
print("Carpeta 'streaming1/ml_model' creada.")

**Paso 4.2: Generar y guardar un modelo simple**

Ejecuta la siguiente celda de código para generar el modelo y guardarlo. Asegúrate de tener `scikit-learn` y `joblib` instalados en tu entorno de Jupyter. Si no, puedes instalarlos con `!pip install scikit-learn joblib`.

In [ ]:
import joblib
from sklearn.linear_model import LogisticRegression
import numpy as np

# Datos dummy para entrenar un modelo simple
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8]])
y = np.array([0, 0, 0, 1, 1, 1, 1])

model = LogisticRegression()
model.fit(X, y)

# Guardar el modelo en la ruta esperada por el consumidor
joblib.dump(model, 'simple_model.pkl')
print("Modelo simple guardado como streaming1/ml_model/simple_model.pkl")

## 5. Configuración de Dependencias con Poetry

Usaremos Poetry para gestionar las dependencias de Python.

**Paso 5.1: Inicializar Poetry y añadir dependencias**

Abre una terminal (no desde Jupyter) en la carpeta `streaming1/` y ejecuta los siguientes comandos:

```bash
cd streaming1/
poetry init --no-interaction
poetry add kafka-python pandas scikit-learn
```

Esto creará los archivos `pyproject.toml` y `poetry.lock` en tu directorio.

## 6. Implementación del Productor de Datos

El productor generará eventos (simulando datos de sensores o métricas) y los enviará a un tópico de Kafka.

**Paso 6.1: Crear el archivo `producer.py`**

Crea un archivo llamado `producer.py` en la raíz de tu carpeta `streaming1/` y pega el siguiente contenido. Luego, no olvides guardar el archivo.

In [ ]:
%%writefile hands_on_streaming/producer.py
import time
import json
import random
from kafka import KafkaProducer

def serialize_json(obj):
    return json.dumps(obj).encode('utf-8')

# Configuración del productor Kafka
producer = KafkaProducer(
    bootstrap_servers=['localhost:9094'], # Usar el puerto expuesto por Docker Compose
    value_serializer=serialize_json
)

topic_name = 'sensor_data'

print(f"Enviando datos al tópico: {topic_name}")

try:
    for i in range(100):
        sensor_id = random.randint(1, 5)
        temperature = round(random.uniform(20.0, 30.0), 2)
        humidity = round(random.uniform(40.0, 60.0), 2)
        pressure = round(random.uniform(900.0, 1100.0), 2)

        data = {
            'sensor_id': sensor_id,
            'timestamp': time.time(),
            'temperature': temperature,
            'humidity': humidity,
            'pressure': pressure
        }

        producer.send(topic_name, value=data)
        print(f"Sent: {data}")
        time.sleep(1) # Enviar un evento cada segundo

except KeyboardInterrupt:
    print("Deteniendo productor...")
finally:
    producer.close()
    print("Productor cerrado.")

## 7. Implementación del Consumidor y Procesamiento ML en Tiempo Real

El consumidor leerá los eventos del tópico de Kafka, realizará un preprocesamiento básico y usará el modelo de ML para hacer inferencias.

**Paso 7.1: Crear el archivo `consumer.py`**

Crea un archivo llamado `consumer.py` en la raíz de tu carpeta `streaming1/` y pega el siguiente contenido. Luego, no olvides guardar el archivo.

In [ ]:
%%writefile hands_on_streaming/consumer.py
import json
import joblib
from kafka import KafkaConsumer
import pandas as pd
import numpy as np

# Cargar el modelo pre-entrenado
model = joblib.load('ml_model/simple_model.pkl')
print("Modelo de ML cargado exitosamente.")

# Configuración del consumidor Kafka
consumer = KafkaConsumer(
    'sensor_data',
    bootstrap_servers=['localhost:9094'], # Usar el puerto expuesto por Docker Compose
    auto_offset_reset='earliest', # Empieza a leer desde el principio si no hay offset guardado
    enable_auto_commit=True,
    group_id='ml-processing-group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

print("Escuchando mensajes en el tópico 'sensor_data'...")

try:
    for message in consumer:
        data = message.value
        print(f"Received: {data}")

        # Simular preprocesamiento de features
        # Para nuestro modelo simple, usaremos temperatura y humedad como features
        features = np.array([[data['temperature'], data['humidity']]])
        
        # Realizar inferencia
        prediction = model.predict(features)[0]
        prediction_proba = model.predict_proba(features)[0].tolist()

        print(f"  Processed features: {features.tolist()}")
        print(f"  Prediction: {'Anomalía' if prediction == 1 else 'Normal'}")
        print(f"  Prediction probabilities: {prediction_proba}")
        print("-" * 30)

except KeyboardInterrupt:
    print("Deteniendo consumidor...")
finally:
    consumer.close()
    print("Consumidor cerrado.")

## 8. Ejecución del Hands-on

Para ver el flujo de datos en acción, necesitarás abrir dos terminales separadas, ambas navegando a la carpeta `streaming1/`.

**Paso 8.1: Iniciar el Consumidor (Terminal 1)**

En tu primera terminal, navega a la carpeta `streaming1/` y ejecuta el consumidor. Este estará a la espera de mensajes.

```bash
cd streaming1/
poetry run python consumer.py
```

Deberías ver el mensaje: "Escuchando mensajes en el tópico 'sensor_data'..."

**Paso 8.2: Iniciar el Productor (Terminal 2)**

En una **segunda terminal**, también navega a la carpeta `streaming1/` y ejecuta el productor. Este comenzará a enviar datos a Kafka.

```bash
cd streaming1/
poetry run python producer.py
```

Verás cómo el productor envía mensajes en la Terminal 2. Simultáneamente, en la Terminal 1 (donde corre el consumidor), deberías ver cómo cada mensaje es recibido, preprocesado y se realiza una predicción.

¡Felicidades! Has implementado un sistema simple de procesamiento de streaming con Kafka y Python para realizar inferencia de Machine Learning en tiempo real.

## 9. Limpieza

Una vez que hayas terminado con el hands-on, es importante detener los servicios de Docker para liberar recursos.

**Paso 9.1: Detener los contenedores de Docker**

En la carpeta `streaming1/`, ejecuta:

```bash
docker compose down
```

**Paso 9.2: Limpieza completa (opcional)**

Si deseas eliminar también las imágenes de Docker y los volúmenes para una limpieza más profunda, puedes usar:

```bash
docker compose down --rmi all --volumes
```